##Test EKF

In [10]:
import sys
import os 
path = os.getcwd()
path_root = path[:-7]
sys.path.append(str(path_root))
from src.PAS import Patient, disturbances, metrics
from PID import PID
from EKF import EKF


import numpy as np
import pandas as pd
import casadi as cas
from scipy.linalg import block_diag
from bokeh.plotting import figure, show
from bokeh.layouts import row, column
from bokeh.io import output_notebook
import matplotlib.pyplot as plt

output_notebook()

Loading BokehJS ...

## Test EKF

In [12]:


age = 24
height = 170
weight = 55
gender = 1
Ce50p = 5.5
Ce50r = 18.2
gamma = 3
beta = 0.5
E0 = 98
Emax = 95


BIS_param = [Ce50p, Ce50r, gamma, beta, E0, Emax]
George = Patient.Patient(age, height, weight, gender, BIS_param = BIS_param, Random = False)

#init controller
param_opti = pd.read_csv('optimal_parameters.csv')
BIS_cible = 50
up_max = 6.67
ur_max = 16.67
ratio = 2
Kp = float(param_opti.loc[param_opti['ratio']==ratio, 'Kp'])
Ti = float(param_opti.loc[param_opti['ratio']==ratio, 'Ti'])
Td = float(param_opti.loc[param_opti['ratio']==ratio, 'Td'])
PID_controller = PID(Kp = Kp, Ti = Ti, Td = Td,
                  N = 5, Te = 1, umax = max(up_max, ur_max/ratio), umin = 0)

#init state estimator
BIS_param_nominal = BIS_param
George_nominal = Patient.Patient(age, height, weight, gender, BIS_param = BIS_param_nominal, Random = False)
Ap = George_nominal.PropoPK.A
Ar = George_nominal.RemiPK.A
Bp = George_nominal.PropoPK.B
Br = George_nominal.RemiPK.B
A = block_diag(Ap,Ar)
B = np.concatenate((Bp,Br), axis = 1)

estimator = EKF(A, B, Bis_param = BIS_param_nominal, ts = 1)


N_simu = 30*60
BIS = np.zeros(N_simu)
BIS_EKF = np.zeros(N_simu)
MAP = np.zeros(N_simu)
CO = np.zeros(N_simu)
Up = np.zeros(N_simu)
Ur = np.zeros(N_simu)
Xp = np.zeros((4, N_simu))
Xr = np.zeros((4, N_simu))
Xp_EKF = np.zeros((4, N_simu))
Xr_EKF = np.zeros((4, N_simu))
uP = 0
for i in range(N_simu):
    # if i == 100:
    #     print("break")

    uR = min(ur_max,max(0,uP*ratio))
    uP = min(up_max,max(0,uP))
    Bis, Co, Map = George.one_step(uP, uR, noise = False)
    Xp[:,i] =  George.PropoPK.x        
    Xr[:,i] =  George.RemiPK.x          
    dist_bis, dist_map, dist_co = disturbances.compute_disturbances(i, 'null')
    BIS[i] = min(100,Bis) + dist_bis
    MAP[i] = Map[0,0] + dist_map
    CO[i] = Co[0,0] + dist_co 
    Up[i] = uP
    Ur[i] = uR
    #estimation
    X, BIS_EKF[i] = EKF.estimate(uP, BIS[i])
    Xp_EKF[:,i] =  X[:4]       
    Xr_EKF[:,i] =  X[4:]             
    uP = PID_controller.one_step(Bis,BIS_cible)

    
    
    
fig, axs = plt.subplots(8,figsize=(14,16))
for i in range(4):
    axs[i].plot(time, Xp[:,i], '-')
    axs[i].plot(time, Xp_EKF[:,i], '-')
    axs[i].set(xlabel='t', ylabel='$xp_' + str(i+1) + '$')
    plt.grid()
    axs[i+4].plot(time, Xr[:len(time),i], '-')
    axs[i+4].plot(time, Xr_EKF[:len(time),i], '-')
    axs[i+4].set(xlabel='t', ylabel='$xr_' + str(i+1) + '$')
plt.show()


ValueError: could not broadcast input array from shape (8,8) into shape (4,4)